## Importación librerias y dataset

In [1]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns

In [2]:
df=pd.read_csv('data\players_total2.csv')

In [3]:
definiciones=pd.read_csv('data\definiciones_fifa.csv')

In [10]:
definiciones[definiciones['campo']=='ram']

,campo,descripcion
88,ram,player attribute playing as Right Attacking Mi...


In [11]:
definiciones['descripcion'].unique()

array(['unique player ID on sofifa', 'URL of the scraped player',
       'player short name', 'player long name',
       'player preferred positions', 'player current overall attribute',
       'player potential overall attribute', 'player value (in EUR)',
       'player weekly wage (in EUR)', 'player age',
       'player date of birth', 'player height (in cm)',
       'player weight (in kg)', 'club id',
       'club name where the player plays', 'league name of the club', nan,
       'player jersey number in the club',
       'club loaning out the player - if applicable',
       'date when the player joined his current club',
       'player contract expiration date', 'player nationality id',
       'player nationality', 'nation team id',
       'player position in the national team',
       'player jersey number in the national team',
       'player preferred foot', 'player weak foot attribute',
       'player skill moves attribute',
       'player international reputation attribute',

In [8]:
definiciones['campo'].unique()

array(['sofifa_id', 'player_url', 'short_name', 'long_name',
       'player_positions', 'overall', 'potential', 'value_eur',
       'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'club_team_id',
       'club_name', 'league_name',
       'league_level,"league rank of the club (e.g. English Premier League is 1, English League Championship is 2, etc.)"',
       'club_position,"player position in the club (e.g. SUB means substitute, RES means reserve)"',
       'club_jersey_number', 'club_loaned_from', 'club_joined',
       'club_contract_valid_until', 'nationality_id', 'nationality_name',
       'nation_team_id', 'nation_position', 'nation_jersey_number',
       'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'body_type', 'real_face',
       'release_clause_eur', 'player_tags', 'player_traits', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_he

In [17]:
porteros=df[df['player_positions']=='GK']

In [18]:
resto_jugadores=df[df['player_positions']!='GK']

In [15]:
df.player_positions.unique().tolist()

['CF',
 'LW, LM',
 'RM, LM, RW',
 'ST',
 'GK',
 'CM, LW',
 'ST, CF',
 'CM, CDM',
 'LM',
 'LM, RM',
 'CDM, RB, CM',
 'CB',
 'LM, CAM',
 'RM, RW',
 'ST, CF, CAM',
 'CAM, LW',
 'CAM, CM, RM',
 'LM, RM, ST, CAM',
 'RM, CAM, ST, LM',
 'LW',
 'CAM, RW',
 'CB, CDM, CM',
 'CAM, RM',
 'ST, RW',
 'CM, CDM, CAM',
 'CB, RB',
 'CDM, CM',
 'CAM, CF, CM, RM',
 'CDM',
 'CAM, CM, LM',
 'LM, RM, CAM',
 'LM, CAM, RM',
 'CM, CAM',
 'RW, ST',
 'CB, CDM',
 'RB',
 'RM',
 'CAM, CM',
 'RW',
 'CM',
 'CAM, LM, CM',
 'RW, ST, RM',
 'LW, RW',
 'RM, LM',
 'CAM, CM, LM, CDM',
 'LW, CAM',
 'CAM, LM, RM',
 'LW, ST',
 'LB',
 'RW, CAM, ST',
 'CAM',
 'CB, LB',
 'CB, RB, LB',
 'RM, CAM',
 'CAM, LM',
 'LM, RM, CM',
 'LB, CM, CDM, LM',
 'RW, RM',
 'CAM, CF, CM',
 'CF, CAM, ST',
 'CM, CAM, CDM',
 'ST, CF, LW, RW',
 'RB, CB',
 'CAM, LW, CM',
 'CM, CDM, LM',
 'CAM, CF',
 'CF, ST',
 'RW, LW',
 'LB, LM',
 'CB, LB, LM',
 'ST, LW, LM',
 'CF, ST, CAM',
 'CM, RM, RB, CF',
 'RM, RWB',
 'CM, CAM, CDM, ST',
 'RW, CF, RM',
 'CF, RW',
 '